In [1]:
#!pip3 freeze > "../requirements.txt"
#!pip3 install -r "../requirements.txt"  # giving some error
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import copy
import pickle
import datetime as dt
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from pins import board_rsconnect
from collections import Counter

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 


DistributionNotFound: The 'google-cloud-bigquery-storage' distribution was not found and is required by the application

In [2]:
def GBQ_data(query_string):
    client = bigquery.Client('turing-230020')
    query = client.query(query_string)
    results = query.result()
    return results.to_dataframe()


In [3]:
job_mcq_query = """SELECT job_id, count(distinct skill_id) as num_skills, count(distinct mcq_id) as num_mcqs,
count(distinct  case when job_skill_level_id=1 then skill_id else NULL end) num_must_have_skills,
count(distinct  case when job_skill_level_id=2 then skill_id else NULL end) num_nice_have_skills,
count(distinct  case when job_skill_level_id=1 and mcq_id is not NULL then skill_id else NULL end) vet_num_must_have_skills,
count(distinct  case when job_skill_level_id=2 and mcq_id is not NULL then skill_id else NULL end) vet_num_nice_have_skills,
count(distinct  case when job_skill_level_id=1 and mcq_id is NULL then skill_id else NULL end) n_vet_num_must_have_skills,
count(distinct  case when job_skill_level_id=2 and mcq_id is NULL then skill_id else NULL end) n_vet_num_nice_have_skills,
string_agg(distinct case when job_skill_level_id=1 then cast(mcq_id as string) else null end order by case when job_skill_level_id=1 then cast(mcq_id as string) else null end) must_have_mcq_id,
string_agg(distinct case when job_skill_level_id=2 then cast(mcq_id as string) else null end order by case when job_skill_level_id=2 then cast(mcq_id as string) else null end) nice_have_mcq_id,
string_agg(distinct case when job_skill_level_id=1 then cast(skill_id as string) else NULL end order by case when job_skill_level_id=1 then cast(skill_id as string) else NULL end) must_have_skill_ids,
string_agg(distinct case when job_skill_level_id=2 then cast(skill_id as string) else NULL end order by case when job_skill_level_id=2 then cast(skill_id as string) else NULL end) nice_have_skill_ids,
string_agg(distinct case when job_skill_level_id=1 and mcq_id is null then cast(skill_id as string) else NULL end order by case when job_skill_level_id=1 and mcq_id is null then cast(skill_id as string) else NULL end) n_vet_must_have_skill_ids,
string_agg(distinct case when job_skill_level_id=2 and mcq_id is null then cast(skill_id as string) else NULL end order by case when job_skill_level_id=2 and mcq_id is null then cast(skill_id as string) else NULL end) n_vet_nice_have_skill_ids,
string_agg(distinct case when job_skill_level_id=1 then skill_name else NULL end) must_have_skill_names,
string_agg(distinct case when job_skill_level_id=2 then skill_name else NULL end) nice_have_skill_names,
string_agg(distinct case when job_skill_level_id=1 and mcq_id is not null then skill_name else NULL end) vet_must_have_skill_names,
string_agg(distinct case when job_skill_level_id=2 and mcq_id is not null then skill_name else NULL end) vet_nice_have_skill_names,
string_agg(distinct case when job_skill_level_id=1 and mcq_id is null then skill_name else NULL end) n_vet_must_have_skill_names,
string_agg(distinct case when job_skill_level_id=2 and mcq_id is null then skill_name else NULL end) n_vet_nice_have_skill_names
from(
SELECT job.job_id, job.skill_id, job.job_skill_level_id, skill.skill_name, ch_name.challenge_name , mcq_id
from `turing-230020.devdb_mirror.ms2_job_skill` job
LEFT join `turing-230020.devdb_mirror.base_all_skills_v4` skill on job.skill_id=skill.id
Left JOIN `turing-230020.devdb_mirror.dv2_skill_mcq` skill_mcq on job.skill_id=skill_mcq.skill_id
left join `turing-230020.devdb_mirror.dv2_challenge` ch_name on skill_mcq.mcq_id=ch_name.challenge_id 
) GROUP by 1
"""

job_mcq = GBQ_data(job_mcq_query)
job_mcq.shape

NameError: name 'bigquery' is not defined

In [4]:
B1 = pd.read_csv('data/must-have-passed-2022-08-15.csv', index_col=0)
B2 = pd.read_csv('data/must-have-self-2022-08-15.csv', index_col=0)
B3 = pd.read_csv('data/nice-have-passed-2022-08-15.csv', index_col=0)
B4 = pd.read_csv('data/nice-have-self-2022-08-15.csv', index_col=0)

B1.shape , B2.shape , B3.shape ,B4.shape

NameError: name 'pd' is not defined

In [5]:
job_mcq_f = job_mcq[['job_id', 'num_skills', 'num_mcqs', 'num_must_have_skills', 'num_nice_have_skills', 'must_have_skill_ids', 'nice_have_skill_ids', 'must_have_skill_names', 'nice_have_skill_names']]
job_mcq_f = job_mcq_f.merge(B1, how='left', on='must_have_skill_ids').merge(B2, how='left', on='must_have_skill_ids')
job_mcq_f = job_mcq_f.merge(B3, how='left', on='nice_have_skill_ids').merge(B4, how='left', on='nice_have_skill_ids')
job_mcq_f.loc[job_mcq_f['must_have_skill_names'].isna(), ['n_devs_must_have_mcq', 'elite_devs_must_have_mcq', 'n_devs_must_have_self', 'elite_devs_must_have_self']] = job_mcq_f.loc[job_mcq_f['must_have_skill_names'].isna(), ['n_devs_must_have_mcq', 'elite_devs_must_have_mcq', 'n_devs_must_have_self', 'elite_devs_must_have_self']].fillna(0)
job_mcq_f.loc[job_mcq_f['nice_have_skill_names'].isna(), ['n_devs_nice_have_mcq', 'elite_devs_nice_have_mcq', 'n_devs_nice_have_self', 'elite_devs_nice_have_self']] = job_mcq_f.loc[job_mcq_f['nice_have_skill_names'].isna(), ['n_devs_nice_have_mcq', 'elite_devs_nice_have_mcq', 'n_devs_nice_have_self', 'elite_devs_nice_have_self']].fillna('No Nice have skill')
fill_na_0 = ['n_devs_must_have_mcq', 'elite_devs_must_have_mcq', 'n_devs_must_have_self', 'elite_devs_must_have_self','n_devs_nice_have_mcq','elite_devs_nice_have_mcq','n_devs_nice_have_self','elite_devs_nice_have_self']
job_mcq_f[fill_na_0] = job_mcq_f[fill_na_0].fillna(0)

In [6]:
all_turing_metrics = """SELECT * except(rn) from (
Select *, row_number() over(partition by job_id order by opportunity_created_date desc) as rn
from `turing-230020.matchingmetrics.all_turing_metrics`
) WHERE rn=1"""
matching_metrics = GBQ_data(all_turing_metrics)
print(matching_metrics.shape)


ms2_job_query = """SELECT job.*, status.status, role.role_type, pr.priority 
from `turing-230020.devdb_mirror.ms2_job` job
left join `turing-230020.devdb_mirror.ms2_job_status` status on status.id = job.job_status_id
left join `turing-230020.devdb_mirror.ms2_job_role_type` role on job.role_type_id = role.id
left join `turing-230020.devdb_mirror.ms2_job_priority` pr on job.priority_id = pr.id;"""

ms2_job = GBQ_data(ms2_job_query)
ms2_job.shape

(8118, 71)


(9414, 58)

In [7]:
job_mcq_f.head(2)

metrics = ['slp_date', 'pcp_date',  'psp_date', 'ir_date', 'ih_date', 'dc_date']
metrics_bin = ['slp_date_bin', 'pcp_date_bin',  'psp_date_bin', 'ir_date_bin', 'ih_date_bin', 'dc_date_bin']
for col in metrics:
    matching_metrics[col+'_bin'] = np.where(matching_metrics[col].isna(),0,1)
    
mm_cols = ['opportunityid', 'job_id', 'opportunity_created_date', 'client_type' , 'client_category', 'region',
          'value', 'chosen_type', 'slp_date', 'pcp_date',  'psp_date', 'ir_date', 'ih_date', 'dc_date'] + metrics_bin

ms2_job_cols = ['id', 'company', 'created_date', 'job_match_eta', 'is_deleted', 'max_acceptable_rate', 'country_id', 'customer_start_date',
              'customer_weekly_hour_engagement', 'number_of_open_roles', 'industry', 'customer_category',
              'company_size', 'must_have_total_years_of_experience', 'job_value', 'opportunity_status',
               'status', 'role', 'role_type', 'priority']

job_data = ms2_job[ms2_job_cols].merge(job_mcq_f, left_on='id', right_on='job_id', how='left')
job_data = job_data.merge(matching_metrics[mm_cols], left_on='id', right_on='job_id', how='left')

In [18]:
ml_cat_col = ['client_type', 'client_category', 'region', 'role_type', 'priority']

ml_num_col = ['num_skills', 'num_mcqs', 'num_must_have_skills', 'num_nice_have_skills',
             'n_devs_must_have_mcq', 'n_devs_nice_have_mcq',
                'n_devs_must_have_self', 'n_devs_nice_have_self','job_value',
             'customer_weekly_hour_engagement', 'is_deleted']


metric = ['dc_date_bin']

ml_data = job_data[['id']+ml_cat_col+ml_num_col+metric].copy()
ml_data_v2 = ml_data.loc[~((ml_data['dc_date_bin'].isna()) | (ml_data['num_skills'].isna())) ,].copy()



In [19]:
ml_data_v2.to_csv('processed-data/process-data-2022-08-15.csv')